# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [51]:
import gym
import random
import numpy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [52]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()

100%|██████████| 10/10 [00:03<00:00,  2.51it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [57]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
        # initialise layers here
        self.layer1 = nn.Linear(observation_size, 10)    # create dense layer 1
        self.layer2 = nn.Linear(10, action_size)     # create dense layer 2
 
    def forward(self, x):
        x = torch.from_numpy(x)
        # send x through the network
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        
        return x

    def predict(self, x):
        x = self.forward(x)            # send x through neural net
        return torch.argmax(x, dim=1)  # predict most likely thing


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [61]:
class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        # memory that stores N most new transitions
        self.memory_size = 1000
        self.memory = [None]*self.memory_size 
        self.last_inserted = 0
        self.memory_full = False
        
        # good place to store hyperparameters as attributes
        self.gamma = 0.9

    def remember(self, state, action, reward, next_state, done):
        sars = {"state": state,
                "action": action,
                "reward": reward,
                "next_state": next_state,
                "done": done}
        # add to memory
        self.memory[self.last_inserted] = sars
        self.last_inserted = (self.last_inserted + 1) % self.memory_size
        if self.last_inserted==0:
            self.memory_full = True

    def act(self, state):
        # return an action from the model
        out = self.model.forward(state)
        return torch.argmax(out, dim=1)

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        if self.memory_full:
            batch = random.sample(self.memory, batch_size)
        else:
            batch = random.sample(self.memory[:batch_size], batch_size)
        for sars in batch:
            self.train(sars)

    def train(self, sars):
        self.optimizer.zero_grad()              # clean gradients of parameters
        pred = self.model.predict(sars.get("state"))    # make prediction
        if sars.get("done") == True:
            y = sars.get("reward")
        else:
            out_tensor = self.model.forward(sars.get("next_state"))
            max_value = torch.max(out_tensor).item()
            y = sars.get("reward") + self.gamma*max_value
        loss = self.criterion(pred, y)          # calculate loss with respect to prediction
        loss.backward()                         # calculate gradients of model.parameters() with respect to loss
        self.optimizer.step()                   # update parameters with respect to gradients



### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [62]:
def train(env, agent: Agent, episodes=1000, batch_size=64):  # train for many games
    for _ in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            tradeoff = random.uniform(0,1)
            if tradeoff > 0.3:
                action = agent.model.predict(state)
            else:
                action = env.action_space.sample()
            # Take the action (a) and observe the outcome state(s') and reward (r)
            new_state, reward, done, _, _ = env.step(action)

            # 2. have the agent remember stuff.
            agent.remember(state, action, reward, new_state, done)

            # 3. update state
            state = new_state

            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
    env.close()

### Putting it together
---
We train an agent on the environment:


In [63]:
env = gym.make('CartPole-v1')  # , render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
torch.save(agent.model.state_dict(), 'model.pth')

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Numpy is not available

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/